## 步驟 1: 環境設定與授權

請在此處選擇您的操作模式、程式碼分支和啟動模式，然後點擊「確認設定並掛載雲端硬碟」。
- **操作模式**: 
  - `暫存模式`: 所有資料（包括資料庫）將在 Colab 執行階段結束後消失。AI 分析功能可能受限（若未設定API金鑰）。
  - `持久模式`: 需要授權 Google Drive，應用程式資料將儲存在您的雲端硬碟中 (位於 `/content/drive/MyDrive/wolfAI_Data`)，並在不同執行階段間保持。
- **GitHub 分支**: 指定要從哪個程式碼分支部署。
- **啟動模式**: 
  - `一般模式`: 以標準方式啟動服務。
  - `除錯模式`: 啟動服務時將輸出更詳細的日誌，方便排查問題。

In [ ]:
# --- Python 模組導入 ---
import os
import shutil
import time
import requests
import subprocess
import pytz
import json # 用於讀取版本資訊
from datetime import datetime
from IPython.display import display, HTML, clear_output, Markdown
import ipywidgets as widgets
from google.colab import output, drive 

# --- 全域狀態變數 ---
settings_confirmed_success = False
services_attempted_start = False 
port_conflict_occurred_global = False 
backend_actually_ready_global = False 
launch_mode_value = "normal" 
operation_mode_value = "transient" 
branch_name_global = "main" 
APP_VERSION = ""

# --- 全域設定 ---
GIT_REPO_URL = "https://github.com/hsp1234-web/wolfAI_v1.git"
PROJECT_PARENT_DIR = "/content/wolf_project"
PROJECT_DIR_NAME = "wolfAI_v1"
FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME)
TAIPEI_TZ = pytz.timezone('Asia/Taipei')
PERSISTENT_DATA_PATH = "/content/drive/MyDrive/wolfAI_Data" 
VERSION_FILE_PATH = os.path.join(FULL_PROJECT_PATH, 'version.json')

# --- UI 元件定義 ---
style = {'description_width': 'initial'}
operation_mode_selection = widgets.Dropdown(
    options=[
        ('暫存模式 (資料與設定不跨執行階段保存)', 'transient'),
        ('持久模式 (資料與設定保存於Google Drive)', 'persistent')
    ],
    value='transient',
    description='1. 操作模式:',
    style=style
)
branch_selection = widgets.Text(
    value='main',
    description='2. GitHub 分支:',
    style=style
)
launch_mode_selection = widgets.RadioButtons(
    options=['一般模式 (normal)', '除錯模式 (debug)'],
    description='3. 啟動模式:',
    disabled=False,
    style=style
)
confirm_settings_button = widgets.Button(
    description="確認設定並準備環境",
    disabled=False,
    button_style='info',
    tooltip='點擊此按鈕以確認以上設定並準備環境。',
    icon='check'
)
settings_output = widgets.Output()
version_output = widgets.Output() # 用於顯示版本資訊

# --- Helper 函數 ---
def get_taipei_time_str():
    return datetime.now(TAIPEI_TZ).strftime('%Y-%m-%d %H:%M:%S %Z')

def load_app_version():
    global APP_VERSION
    try:
        if os.path.exists(VERSION_FILE_PATH):
            with open(VERSION_FILE_PATH, 'r') as f:
                version_data = json.load(f)
                APP_VERSION = version_data.get('version', '未知')
        else:
            APP_VERSION = "未知 (檔案未找到)"
    except Exception as e:
        APP_VERSION = f"未知 (讀取錯誤: {e})"
    with version_output:
        clear_output(wait=True)
        display(Markdown(f"**應用程式版本:** {APP_VERSION}"))

def run_shell_command(command, cwd=None, capture_output=False):
    if capture_output:
        process = subprocess.run(command, shell=True, text=True, capture_output=True, cwd=cwd)
        return process
    else:
        print(f"[{get_taipei_time_str()}] Executing: {command}")
        process = subprocess.Popen(
            command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            shell=True, text=True, bufsize=1, universal_newlines=True, cwd=cwd
        )
        for line in iter(process.stdout.readline, ''):
            print(line, end='')
        process.stdout.close()
        return_code = process.wait()
        if return_code != 0:
            print(f"[{get_taipei_time_str()}] Command failed with code {return_code}")
        return return_code 

# --- 按鈕點擊事件處理函數 ---
def on_confirm_settings_clicked(b):
    global settings_confirmed_success, launch_mode_value, operation_mode_value, branch_name_global
    settings_confirmed_success = False 
    with settings_output:
        clear_output(wait=True)
        confirm_settings_button.disabled = True
        operation_mode_selection.disabled = True
        branch_selection.disabled = True
        launch_mode_selection.disabled = True

        operation_mode_value = operation_mode_selection.value
        launch_mode_value = launch_mode_selection.value.split('(')[1].replace(')', '')
        branch_name_global = branch_selection.value.strip()

        print(f"[{get_taipei_time_str()}] ✅ 設定已確認")
        op_mode_label = dict(operation_mode_selection.options).get(operation_mode_value, '未知模式')
        print(f"   - 操作模式: {operation_mode_value} ({op_mode_label}) ")
        print(f"   - 啟動模式: {launch_mode_value}")
        print(f"   - 程式碼分支: {branch_name_global}")
        print("-" * 50)

        os.environ['OPERATION_MODE'] = operation_mode_value
        os.environ['LAUNCH_MODE'] = launch_mode_value 
        os.environ['GIT_BRANCH_NAME'] = branch_name_global

        print(f"\n[{get_taipei_time_str()}] 正在處理操作模式設定...")
        if operation_mode_value == "persistent":
            print(f"[{get_taipei_time_str()}] 持久模式：嘗試掛載 Google Drive...")
            try:
                drive.mount('/content/drive', force_remount=True)
                print(f"[{get_taipei_time_str()}] ✅ Google Drive 已掛載於 /content/drive。")
                os.makedirs(PERSISTENT_DATA_PATH, exist_ok=True)
                print(f"[{get_taipei_time_str()}] 應用程式資料將儲存於: {PERSISTENT_DATA_PATH}")
                os.environ['REPORTS_DB_PATH'] = os.path.join(PERSISTENT_DATA_PATH, 'reports.sqlite')
                os.environ['PROMPTS_DB_PATH'] = os.path.join(PERSISTENT_DATA_PATH, 'prompts.sqlite')
                print(f"[{get_taipei_time_str()}] 持久模式下資料庫路徑已設定。")
                print(f"[{get_taipei_time_str()}] 提示: 請確保已在 Colab Secrets 中設定持久模式所需金鑰。")
                settings_confirmed_success = True
            except Exception as e:
                print(f"[{get_taipei_time_str()}] ❌ 掛載 Google Drive 失敗: {e}")
                settings_confirmed_success = False
        else:
            print(f"[{get_taipei_time_str()}] 暫存模式：跳過 Google Drive 掛載。")
            os.environ.pop('REPORTS_DB_PATH', None) 
            os.environ.pop('PROMPTS_DB_PATH', None) 
            if not os.getenv('COLAB_GOOGLE_API_KEY'):
                print(f"[{get_taipei_time_str()}] ⚠️ 警告: 未偵測到 Colab Secret 'COLAB_GOOGLE_API_KEY'。AI 功能將受限。")
            settings_confirmed_success = True
        
        if settings_confirmed_success:
            print(f"\n[{get_taipei_time_str()}] ✅ 環境設定完成。請執行下一步驟。")
        else:
            print(f"\n[{get_taipei_time_str()}] ❌ 環境設定未成功。請修正錯誤後重試。")

        confirm_settings_button.disabled = False
        operation_mode_selection.disabled = False
        branch_selection.disabled = False
        launch_mode_selection.disabled = False

confirm_settings_button.on_click(on_confirm_settings_clicked)
display(version_output) # 初始顯示版本區域
display(
    widgets.VBox([
        operation_mode_selection,
        branch_selection,
        launch_mode_selection,
        confirm_settings_button
    ]),
    settings_output
)
# 首次執行時嘗試載入版本（如果專案已存在且 version.json 可用）
# load_app_version() # 這行會太早執行，移到步驟2 clone/pull 之後


## 步驟 2: 安裝依賴與啟動服務

點擊下方按鈕開始下載專案程式碼、安裝所需依賴，並啟動後端與前端服務。
**注意**: 此步驟可能需要幾分鐘時間，特別是首次執行時。

In [ ]:
start_services_button = widgets.Button(
    description="開始安裝與啟動服務",
    disabled=False,
    button_style='primary',
    tooltip='點擊此按鈕開始下載、安裝並啟動應用程式服務。',
    icon='cogs'
)
services_output_step2 = widgets.Output() # 與步驟1的 output 分開
stop_services_button = widgets.Button(
    description="停止所有服務",
    disabled=True, 
    button_style='danger',
    tooltip='嘗試停止由啟動腳本啟動的背景進程。',
    icon='stop'
)

def on_start_services_clicked(b):
    global services_attempted_start, port_conflict_occurred_global, backend_actually_ready_global
    services_attempted_start = True
    port_conflict_occurred_global = False
    backend_actually_ready_global = False

    with services_output_step2:
        clear_output(wait=True)
        start_services_button.disabled = True
        stop_services_button.disabled = True 

        if not settings_confirmed_success:
            print(f"[{get_taipei_time_str()}] ❌ 錯誤：請先完成步驟 1 的「環境設定」並確保其成功完成。")
            start_services_button.disabled = False
            return

        current_launch_mode = launch_mode_value
        current_branch_name = branch_name_global

        print(f"[{get_taipei_time_str()}] 步驟 2.1: 複製/更新專案程式碼 (分支: {current_branch_name})...")
        project_git_dir = os.path.join(FULL_PROJECT_PATH, '.git')

        if os.path.exists(FULL_PROJECT_PATH) and os.path.exists(project_git_dir):
            print(f"[{get_taipei_time_str()}] 偵測到已存在的專案目錄，嘗試更新...")
            status_check = run_shell_command(f'cd {FULL_PROJECT_PATH} && git status -sb', capture_output=True)
            current_local_branch = ''
            if status_check.returncode == 0 and status_check.stdout:
                current_local_branch = status_check.stdout.split('\n')[0].replace('## ', '').split('...')[0]
            
            print(f"[{get_taipei_time_str()}] 目前本機分支: '{current_local_branch}', 目標分支: '{current_branch_name}'")
            if current_local_branch != current_branch_name:
                print(f"[{get_taipei_time_str()}] 分支不符，執行切換分支到 {current_branch_name}...")
                run_shell_command(f'cd {FULL_PROJECT_PATH} && git stash push -u', cwd=FULL_PROJECT_PATH)
                checkout_rc = run_shell_command(f'cd {FULL_PROJECT_PATH} && git checkout {current_branch_name} && git pull origin {current_branch_name} --ff-only', cwd=FULL_PROJECT_PATH)
                if checkout_rc != 0:
                    print(f"[{get_taipei_time_str()}] ❌ 切換/拉取失敗。請檢查 Git 狀態。")
            else:
                print(f"[{get_taipei_time_str()}] 已在分支 {current_branch_name}，拉取最新程式碼...")
                run_shell_command(f'cd {FULL_PROJECT_PATH} && git stash push -u', cwd=FULL_PROJECT_PATH) 
                pull_rc = run_shell_command(f'cd {FULL_PROJECT_PATH} && git pull origin {current_branch_name} --ff-only', cwd=FULL_PROJECT_PATH)
                if pull_rc != 0:
                     print(f"[{get_taipei_time_str()}] ❌ 拉取最新程式碼失敗。請檢查 Git 狀態。")
            # 嘗試恢復 Stash (如果有的話)
            # run_shell_command(f'cd {FULL_PROJECT_PATH} && git stash pop', cwd=FULL_PROJECT_PATH) # 暫不自動 pop
        else:
            print(f"[{get_taipei_time_str()}] 未偵測到專案或非 Git 倉庫，執行全新複製...")
            if os.path.exists(FULL_PROJECT_PATH):
                 shutil.rmtree(FULL_PROJECT_PATH) 
            clone_rc = run_shell_command(f"git clone --depth 1 -b {current_branch_name} {GIT_REPO_URL} {FULL_PROJECT_PATH}")
            if clone_rc != 0:
                 print(f"[{get_taipei_time_str()}] ❌ 嚴重錯誤: 專案複製失敗。")
                 start_services_button.disabled = False
                 return
        
        if not os.path.exists(FULL_PROJECT_PATH):
            print(f"[{get_taipei_time_str()}] ❌ 嚴重錯誤: 專案目錄在複製/更新後不存在。")
            start_services_button.disabled = False
            return
        print(f"[{get_taipei_time_str()}] ✅ 專案已成功準備於 {FULL_PROJECT_PATH}。")
        load_app_version() # 更新/複製後載入版本號

        print(f"\n[{get_taipei_time_str()}] 步驟 2.2: 執行啟動腳本 (scripts/colab_start.sh) 以模式: {current_launch_mode}...")
        start_script_path = os.path.join(FULL_PROJECT_PATH, 'scripts', 'colab_start.sh')
        if not os.path.exists(start_script_path):
            print(f"[{get_taipei_time_str()}] ❌ 嚴重錯誤: 找不到啟動腳本 {start_script_path}。")
            start_services_button.disabled = False
            return
        os.chmod(start_script_path, 0o755)
        
        process = run_shell_command(
            f'bash {start_script_path} --mode={current_launch_mode}', 
            cwd=FULL_PROJECT_PATH, 
            capture_output=True 
        )
        
        pid_file_path = os.path.join(FULL_PROJECT_PATH, "colab_pids.txt")
        log_file_path = os.path.join(FULL_PROJECT_PATH, "colab_execution.log")

        if process.stdout:
            print(f"[{get_taipei_time_str()}] --- 啟動腳本 STDOUT ---\n{process.stdout}")
        if process.stderr:
            print(f"[{get_taipei_time_str()}] --- 啟動腳本 STDERR ---\n{process.stderr}")

        port_conflict_in_log = False
        try:
            if os.path.exists(log_file_path):
                with open(log_file_path, 'r') as log_f:
                    if "ERROR_PORT_IN_USE_請注意端口已被佔用" in log_f.read():
                        port_conflict_in_log = True
        except Exception as e_log_read:
             print(f"[{get_taipei_time_str()}] 讀取啟動日誌時發生錯誤: {e_log_read}")

        if process.returncode == 10 or port_conflict_in_log:
            port_conflict_occurred_global = True
            print(f"[{get_taipei_time_str()}] ❌ 偵測到端口衝突。請參閱步驟 3。")
        elif process.returncode != 0:
            print(f"[{get_taipei_time_str()}] ❌ 警告: 啟動腳本異常退出 (碼: {process.returncode})。健康檢查可能失敗。")
        else:
            print(f"[{get_taipei_time_str()}] ✅ 啟動腳本已執行。")
            if os.path.exists(pid_file_path):
                print(f"[{get_taipei_time_str()}] PID 文件 {pid_file_path} 已創建。")
                stop_services_button.disabled = False 
            else:
                print(f"[{get_taipei_time_str()}] ⚠️ PID 文件 {pid_file_path} 未找到。服務可能未啟動。")

        print(f"\n[{get_taipei_time_str()}] ✅ 啟動流程完成。請執行下一步驟進行健康檢查。")
        start_services_button.disabled = False

def on_stop_services_clicked(b):
    with services_output_step2: # 確保輸出到正確的 Output widget
        clear_output(wait=True)
        stop_services_button.disabled = True
        print(f"[{get_taipei_time_str()}] 正在嘗試停止服務...")
        
        stop_script_path = os.path.join(FULL_PROJECT_PATH, 'scripts', 'colab_stop.sh')
        if not os.path.exists(stop_script_path):
            print(f"[{get_taipei_time_str()}] ❌ 錯誤: 找不到停止腳本 {stop_script_path}。")
            start_services_button.disabled = False 
            return
        
        os.chmod(stop_script_path, 0o755)
        process = run_shell_command(f'bash {stop_script_path}', cwd=FULL_PROJECT_PATH, capture_output=True)
        
        if process.stdout:
            print(f"[{get_taipei_time_str()}] --- 停止腳本 STDOUT ---\n{process.stdout}")
        if process.stderr:
            print(f"[{get_taipei_time_str()}] --- 停止腳本 STDERR ---\n{process.stderr}")

        if process.returncode == 0:
            print(f"[{get_taipei_time_str()}] ✅ 停止腳本執行成功。服務應已停止。")
            global services_attempted_start, backend_actually_ready_global, port_conflict_occurred_global
            services_attempted_start = False
            backend_actually_ready_global = False
            port_conflict_occurred_global = False
            pid_file_path = os.path.join(FULL_PROJECT_PATH, "colab_pids.txt")
            if os.path.exists(pid_file_path):
                try: os.remove(pid_file_path); print(f"[{get_taipei_time_str()}] 已移除 PID 文件。");
                except Exception as e: print(f"[{get_taipei_time_str()}] 移除 PID 文件出錯: {e}")
        else:
            print(f"[{get_taipei_time_str()}] ⚠️ 停止腳本異常退出 (碼: {process.returncode})。")
        
        start_services_button.disabled = False 

start_services_button.on_click(on_start_services_clicked)
stop_services_button.on_click(on_stop_services_clicked)
display(widgets.HBox([start_services_button, stop_services_button]), services_output_step2)


## 步驟 3: 健康檢查與獲取訪問連結

點擊下方按鈕以檢查後端和前端服務的狀態。如果服務均已就緒，將會顯示前端的訪問連結。
**注意**: 健康檢查可能需要 1-2 分鐘，請耐心等待。

In [ ]:
health_check_button = widgets.Button(
    description="執行健康檢查並獲取連結",
    disabled=False,
    button_style='success',
    tooltip='點擊以檢查服務狀態並獲取前端訪問連結。',
    icon='medkit'
)
health_check_output = widgets.Output()

PORT_ERROR_HTML = """
<div style="border: 2px solid red; padding: 10px; background-color: #ffebee;">
<h3>⚠️ 應用程式啟動失敗！</h3>
<p><b>原因：</b>網路端口已被占用。這通常發生在 Colab 環境中，當上一個執行階段未完全釋放端口時。</p>
<p><b>解決方案：</b></p>
<ol>
  <li>嘗試點擊步驟 2 中的「停止所有服務」按鈕，等待幾秒鐘，然後重新點擊「開始安裝與啟動服務」。</li>
  <li>如果問題仍然存在，請點選上方選單的【執行階段】->【中斷並重新啟動執行階段】，然後從步驟 1 開始重新執行所有儲存格。</li>
</ol>
</div>
"""

GENERAL_ERROR_HTML_TEMPLATE = """
<h2>❌ 應用程式未能成功啟動。</h2>
<p>請檢查 Cell 2 的日誌輸出 (以及位於專案目錄下 <code>colab_execution.log</code> 的詳細日誌) 以了解詳細錯誤。</p>
<p><b>版本:</b> {version}</p>
<p>您可以嘗試重新啟動執行階段並再次執行所有步驟。</p>
<p>如果問題持續，請考慮使用「除錯模式」以獲取更詳細的日誌輸出，並檢查 Colab Secret 中的 API 金鑰設定是否正確。</p>
"""

def on_health_check_clicked(b):
    global backend_actually_ready_global
    backend_actually_ready_global = False

    with health_check_output:
        clear_output(wait=True)
        health_check_button.disabled = True
        load_app_version() # 確保顯示最新的版本資訊
        display(version_output)

        if not services_attempted_start:
            print(f"[{get_taipei_time_str()}] ⚠️ 提示：請先執行步驟 2 的「安裝依賴與啟動服務」。")
            health_check_button.disabled = False
            return

        if port_conflict_occurred_global:
            display(HTML(PORT_ERROR_HTML))
            print(f"[{get_taipei_time_str()}] 由於偵測到端口衝突，健康檢查未執行。")
            health_check_button.disabled = False
            return
        
        print(f"[{get_taipei_time_str()}] --- 後端健康檢查開始 ---")
        backend_health_url = "http://localhost:8000/api/health"
        max_wait_seconds_backend = 120 
        poll_interval = 10 
        start_time_backend = time.time()
        backend_ready = False

        while time.time() - start_time_backend < max_wait_seconds_backend:
            print(f"[{get_taipei_time_str()}] 正在檢查後端 ({backend_health_url})...")
            try:
                response = requests.get(backend_health_url, timeout=8)
                if response.status_code == 200:
                    health_data = response.json()
                    print(f"[{get_taipei_time_str()}] ✅ 後端回應: {health_data.get('message', 'OK')}")
                    if health_data.get('status') == '正常' or health_data.get('status') == '警告':
                         backend_ready = True
                         backend_actually_ready_global = True
                         break
                    else:
                        print(f"[{get_taipei_time_str()}] ⚠️ 後端狀態 '{health_data.get('status')}', 訊息: {health_data.get('message')}")
                else:
                    print(f"[{get_taipei_time_str()}] ⏳ 後端返回 {response.status_code}。重試...")
            except requests.exceptions.ConnectionError:
                print(f"[{get_taipei_time_str()}] ⏳ 後端連線失敗。重試...")
            except requests.exceptions.Timeout:
                print(f"[{get_taipei_time_str()}] ⏳ 後端請求超時。重試...")
            except Exception as e:
                print(f"[{get_taipei_time_str()}] ⏳ 健康檢查錯誤: {e}。重試...")
            time.sleep(poll_interval)
        
        if not backend_ready:
            print(f"\n[{get_taipei_time_str()}] ❌ 後端服務在 {max_wait_seconds_backend} 秒內未能啟動或不健康。")
            display(HTML(GENERAL_ERROR_HTML_TEMPLATE.format(version=APP_VERSION)))
            log_file_path = os.path.join(FULL_PROJECT_PATH, "colab_execution.log")
            try:
                if os.path.exists(log_file_path):
                    with open(log_file_path, 'r') as log_f:
                        lines = log_f.readlines()
                        last_n_lines = "".join(lines[-20:]) 
                        print(f"\n--- colab_execution.log (最後 20 行) ---\n{last_n_lines}")
            except Exception as e_log:
                print(f"[{get_taipei_time_str()}] 讀取 colab_execution.log 錯誤: {e_log}")
            health_check_button.disabled = False
            return
        
        print(f"\n[{get_taipei_time_str()}] --- 前端服務檢查開始 ---")
        frontend_ready = False
        frontend_url = ""
        try:
            frontend_url = output.eval_js(f'google.colab.kernel.proxyPort(3000, {{"cache_bust": True}})')
            print(f"[{get_taipei_time_str()}] 前端代理 URL: {frontend_url}")
        except Exception as e:
            print(f"[{get_taipei_time_str()}] ⚠️ 無法獲取前端 Colab 代理 URL: {e}")
            display(HTML("<h2>⚠️ 前端檢查跳過</h2><p>無法獲取前端 Colab 代理 URL。</p>"))
            health_check_button.disabled = False
            return

        max_wait_seconds_frontend = 60 
        start_time_frontend = time.time()
        while time.time() - start_time_frontend < max_wait_seconds_frontend:
            print(f"[{get_taipei_time_str()}] 正在檢查前端 ({frontend_url})...")
            try:
                response_frontend = requests.get(frontend_url, timeout=8, headers={'User-Agent': 'ColabHealthCheck/1.0'})
                if response_frontend.status_code == 200:
                    print(f"[{get_taipei_time_str()}] ✅ 前端服務已就緒 ({response_frontend.status_code})。")
                    frontend_ready = True
                    break
                else:
                    print(f"[{get_taipei_time_str()}] ⏳ 前端返回 {response_frontend.status_code}。重試...")
            except requests.exceptions.RequestException as e_req:
                print(f"[{get_taipei_time_str()}] ⏳ 前端請求錯誤: {e_req}。重試...")
            time.sleep(poll_interval)

        border = "="*50
        print(f"\n{border}")
        if backend_ready and frontend_ready:
            success_html = f"<h2>✅ 應用程式已成功啟動！ (版本: {APP_VERSION})</h2><p>🔗 <b>前端訪問網址:</b> <a href='{frontend_url}' target='_blank'>{frontend_url}</a></p><p>如果頁面無法載入，請稍等或檢查網路。</p>"
            display(HTML(success_html))
        elif backend_ready and not frontend_ready:
            display(HTML(f"<h2>⚠️ 後端已就緒，但前端未能確認！(版本: {APP_VERSION})</h2><p>後端服務已啟動，但前端 ({frontend_url}) 未回應。</p><p>嘗試手動訪問：<a href='{frontend_url}' target='_blank'>{frontend_url}</a></p><p>檢查 Cell 2 及 <code>colab_execution.log</code> 的日誌。</p>"))
        else: 
            display(HTML(GENERAL_ERROR_HTML_TEMPLATE.format(version=APP_VERSION)))
        print(f"{border}\n")

        health_check_button.disabled = False

health_check_button.on_click(on_health_check_clicked)
display(health_check_button, health_check_output)
# 首次點擊健康檢查按鈕前，主動載入一次版本資訊（如果步驟二已執行且專案存在）
# 這確保了即使不點擊，如果 version.json 已因步驟二的 git clone/pull 而存在，版本號也能被初步顯示。
# 但更好的做法是，在步驟二 clone/pull 成功後立即調用 load_app_version()。
if os.path.exists(VERSION_FILE_PATH):
    load_app_version()
    display(version_output)
